<a href="https://colab.research.google.com/github/haaden/MSDS_498/blob/main/MSDS498_FINBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import pandas as pd

In [ ]:
import os
import random

import pickle 
import re,string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt

import gensim
from gensim.models import Word2Vec,LdaMulticore, TfidfModel, CoherenceModel
from gensim import corpora
from gensim import similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import corpora
from gensim.models import LsiModel,LdaModel

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_doc(doc): 
    #split document into individual words
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 4]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]         
    # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]
    return tokens

def final_processed_text(doc):
    #this is a function to join the processed text back
    ' '.join(doc)
    return doc

In [ ]:
nw = pd.read_pickle('newsdf.pkl')

In [ ]:
nw.columns

Index(['symbol', 'publishedDate', 'title', 'image', 'site', 'text', 'url'], dtype='object')

In [ ]:
nw = nw.drop(['image','url'], axis=1)

In [ ]:
pressdf= pd.read_pickle('pressdf.pkl')

In [ ]:
pressdf.columns

Index(['symbol', 'date', 'title', 'text'], dtype='object')

In [ ]:
transdf = pd.read_pickle('transcriptdf.pkl')

In [ ]:
transdf.columns

Index(['symbol', 'quarter', 'year', 'date', 'content'], dtype='object')

In [ ]:
#data= data[data["Movie Title"]=="Frozen_2"]
#adding two columns to the dataframe to store the processed text and tokenized text
nw['processed_text'] = nw['text'].apply(lambda x: clean_doc(x))

#creating final processed text variables for matrix creation
nw['final_processed_text'] = [' '.join(x) for x in nw['processed_text']]

In [ ]:
len(pressdf['text'].tolist())

1926

In [ ]:
#data= data[data["Movie Title"]=="Frozen_2"]
#adding two columns to the dataframe to store the processed text and tokenized text
pressdf['processed_text'] = pressdf['text'].apply(lambda x: clean_doc(x))

#creating final processed text variables for matrix creation
pressdf['final_processed_text'] = [' '.join(x) for x in pressdf['processed_text']]

In [ ]:
# cleaning transcripts

In [ ]:
translist = transdf.content.str.split("\n").tolist()

In [ ]:
#transdf['final_processed_text']= [' '.join(x) for x in transdf['text_clean']]

In [ ]:
#transdf= transdf.drop('final_processed_text',axis=1)

In [ ]:
transdf['text_clean'].apply(lambda x: [''.join(a) for a in x]).tolist()

In [ ]:
clean_text= []
for i in translist:
  txt= []
  for a in i:
    doc= clean_doc(a)
    if len(doc)>512:
      docsplit = a.split(".")
      print(docsplit)
      for c in docsplit:
        print(c)
        if len(c)>512:
           doc_split= c.split(",")
           for d in doc_split:
              doc = clean_doc(d)
              txt.append(doc)
        else:    
            doc= clean_doc(c)
            txt.append(doc)
    else:
      txt.append(doc)
    
  clean_text.append(txt)

In [ ]:
final_text=[]
for i in range(len(clean_text)):
  txt=[]
  for a in range(len(clean_text[i])):
    print(clean_text[i])
    txt.append([' '.join(clean_text[i][a])])  #(x) for x in clean_text[i]])
  final_text.append(txt)

In [ ]:
len(final_text)

23

In [ ]:
for i in translist[22]:
  print(len(i))
  if len(i)>512:
    print(i.split("."))

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

In [ ]:
# tested in transformers==4.18.0 
import transformers
transformers.__version__

'4.29.2'

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg',truncation=True, max_length=512)

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer, truncation=True, max_length=512)

In [ ]:
pip install xformers

In [ ]:
# only Q1 news
nw= nw[nw['publishedDate']>'2022-12-31'].copy()

In [ ]:
nw.shape

(1048, 7)

In [ ]:
nw["ESG"]= nw['final_processed_text'].apply(nlp)

In [ ]:
ls= nw['ESG'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
ls

In [ ]:
nw = pd.concat((nw,ls ), axis=1 )

In [ ]:
nw.to_pickle("nw_ESG.pkl")

In [ ]:
#pressdf

Index(['symbol', 'date', 'title', 'text'], dtype='object')

In [ ]:
pressdf["ESG"]= pressdf['final_processed_text'].apply(nlp)

In [ ]:
pls= pressdf['ESG'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
pressdf= pd.concat((pressdf,pls ), axis=1 )

In [ ]:
pressdf.to_pickle('pressESG.pkl')

In [ ]:
final_text[22][3]

In [ ]:
ESG=[]

for i in range(len(clean_text)):
  esg=[]
  for a in clean_text[i]:
    print(len(a))
    if len(a)<=512:
      doc = ' '.join(a)
      esg.append(nlp(doc))
    else:
      a= a[0:512]
      doc = ' '.join(a)
      esg.append(nlp(doc))
  ESG.append(esg)


In [ ]:
pd.DataFrame(ESG).T.to_pickle("TransESG.pkl")

In [ ]:
pd.DataFrame(final_text).T.to_pickle("final_text.pkl")

In [ ]:
pd.DataFrame

In [ ]:
for i in clean_text:
  for a in i:
    if len(a)==488:
      doc= ' '.join(a)
      print(doc)
      #nlp(doc)

thank morning welcome first quarter conference today discuss continued progress executing strategy cover operations rielly review financial results believe offers unique value proposition investors strategy deliver highreturn resource growth supply industryleading growth maintain industry leadership environmental social governance performance disclosure terms resource growth multiple phases guyana developments coming online robust inventory highreturn drilling locations bakken deliver highly profitable production growth annually stabroek block guyana currently sight floating production storage floating vessels fpsos gross production capacity million barrels terms supply resource continues expand steadily curve forecast costs decline approximately portfolio achieve breakeven brent price approximately barrel sanctioned developments stabroek block breakeven brent price approximately barrel terms growth industryleading change story industryleading duration story providing highly differenti

In [ ]:
t= 'thank morning welcome first quarter conference today discuss continued progress executing strategy cover operations rielly review financial results believe offers unique value proposition investors strategy deliver highreturn resource growth supply industryleading growth maintain industry leadership environmental social governance performance disclosure terms resource growth multiple phases guyana developments coming online robust inventory highreturn drilling locations bakken deliver highly profitable production growth annually stabroek block guyana currently sight floating production storage floating vessels fpsos gross production capacity million barrels terms supply resource continues expand steadily curve forecast costs decline approximately portfolio achieve breakeven brent price approximately barrel sanctioned developments stabroek block breakeven brent price approximately barrel terms growth industryleading change story industryleading duration story providing highly differentiated value proposition based brent price barrel barrel forecast increase approximately annually twice topline growth balance sheet continue strengthen recent debttoebitdax ratio approximately onetime successful execution strategy uniquely positioned company deliver significant value shareholders years growing intrinsic value growing returns financial priorities allocate capital highreturn lowcost investment opportunities maintain strong balance sheet position ensure worldclass investment opportunities guyana bakken allocated capital budget return annual shareholders dividend increases share repurchases return capital framework march increased annual dividend share looking ahead continue increasing regular dividend level attractive income oriented investors sustainable price environment generation steadily increases future years share repurchases expected represent growing proportion return capital manage price volatility hedged barrels barrels barrel options barrels barrel brent options positions shareholders protected downside fully benefiting upside strategy guyana industrys largest province discovered decade interest exxonmobil operator since discoveries block including since start fangtooth underpinning gross discovered recoverable resource estimate billion barrels equivalent multibillion barrels exploration potential remaining potential fpsos develop discovered resources block phase phase developments produced average approximately gross barrels first quarter third sanctioned development payara arrived stabroek block earlier month ahead schedule targeted start early fourth quarter gross production capacity approximately fourth sanction development yellowtail expected online gross production capacity approximately barrels government regulatory approvals expected hopefully fifth development gross production capacity approximately barrels development sixth development whiptail expected submitted government regulatory government approvals later turning bakken continue operating fourrig program enable production approximately barrels equivalent lower fully optimize infrastructure generate significant levels continue outstanding applying manufacturing principles build culture innovation improve efficiency mitigate inflationary pressures execute company strategy continue guided longstanding commitment sustainability proud industry leader earlier month announced million donation years institute harnessing plants initiative potential changer tackling global challenge climate change developing plants crops wetlands natural ability capture store potentially billions carbon atmosphere proud received rating latest environmental social governance rating assessment mscis highest rating designates company leader managing industryspecific risks relative peers received first rating earning ratings consecutive years february earned place bloomberg gender equality index fourth consecutive summary continue successfully execute strategy offers unique value proposition industry growing intrinsic value returns multiple phases lowcost developments coming online guyana robust inventory highreturn drilling locations bakken portfolio positioned become increasingly positive continue prioritize return capital shareholders dividend increases share repurchases operational update'

In [ ]:
transdf

In [ ]:
tESG= pd.DataFrame(ESG).T#.columns= transdf['symbol'].tolist()

In [ ]:
tESG.columns=transdf['symbol'].tolist()

In [ ]:
tESG=tESG.T.reset_index(names='symbol')

In [ ]:
#tESG= tESG.stack()#.reset_index()#.drop('level_0', axis=1)

In [ ]:
tESG= pd.melt(tESG, id_vars=["symbol"], 
                   value_name="Value")

In [ ]:
tESG = tESG.dropna()

In [ ]:
tls= tESG['ESG'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
tESGdf= pd.concat((tESG,tls),axis=1) 

In [ ]:
ftxt= pd.DataFrame(final_text).T

In [ ]:
ftxt.columns=transdf['symbol'].tolist()

In [ ]:
ftxt= ftxt.T.reset_index(names='symbol')

In [ ]:
ftxt= pd.melt(ftxt, id_vars=["symbol"], 
                   value_name="Value")

In [ ]:
ftxt.columns= ['symbol',"number","final_text"]

In [ ]:
ftxt= ftxt.dropna()

In [ ]:
ftxt.shape

(3411, 3)

In [ ]:
tESGdf.shape

(3411, 5)

In [ ]:
transtxtESG= ftxt.merge(tESGdf, on=['symbol','number'])

In [ ]:
transESGdf= transdf.merge(transtxtESG, on='symbol')

In [ ]:
transESGdf.to_pickle("transESGdf.pkl")

In [ ]:
transESGdf

In [ ]:
TransESGscore= transESGdf[(transESGdf['score']>0.8) & (transESGdf['label']!='None') ].copy()

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

In [ ]:
TransESGscore["ESG_SCORE"]=TransESGscore["final_text"].apply(nlp)

In [ ]:
tes= TransESGscore["ESG_SCORE"].apply(lambda x: pd.Series(x[0]))

In [ ]:
tes.columns= ['label_s','score_s']

In [ ]:
TransESGfinal= pd.concat((TransESGscore,tes),axis=1)

In [ ]:
TransESGfinal.to_pickle("final_trans.pkl")

In [ ]:
nwESG= pd.read_pickle("nw_ESG.pkl")

In [ ]:
nwESG.head(1)

In [ ]:
nwESGdf= nwESG[(nwESG['label']!='None')&(nwESG['score']>0.8)].copy()

In [ ]:
nwESGdf['ESG_score']=nwESGdf['final_processed_text'].apply(nlp)

In [ ]:
nes = nwESGdf['ESG_score'].apply(lambda x: pd.Series(x[0]))

In [ ]:
nes.columns= ['label_s','score_s']

In [ ]:
nwESGfinal= pd.concat((nwESGdf, nes),axis=1)

In [ ]:
nwESGfinal.to_pickle('nwESGfinal.pkl')

In [ ]:
nwESGfinal.head(1)

,symbol,publishedDate,title,site,text,processed_text,final_processed_text,ESG,label,score,ESG_score,label_s,score_s
11,APA,2023-03-24 07:56:00,Midstream Caught In Middle Of Permian Producer...,Seeking Alpha,Public and private producers are parting ways ...,"[public, private, producers, parting, permian,...",public private producers parting permian basin...,"[{'label': 'Social', 'score': 0.86109918355941...",Social,0.861099,"[{'label': 'Neutral', 'score': 0.9998087286949...",Neutral,0.999809


In [ ]:
pressESG= pd.read_pickle("pressESG.pkl")

In [ ]:
pressESG.head(1)

,symbol,date,title,text,processed_text,final_processed_text,ESG,label,score
0,APA,2023-05-03 16:15:00,APA CORPORATION ANNOUNCES FIRST-QUARTER 2023 F...,"HOUSTON, MAY 03, 2023 (GLOBE NEWSWIRE) -- APA ...","[houston, globe, newswire, corporation, nasdaq...",houston globe newswire corporation nasdaq toda...,"[{'label': 'None', 'score': 0.9842051863670349}]",None,0.984205


In [ ]:
pressESGdf = pressESG[(pressESG['label']!='None')&(pressESG['score']>0.8)].copy()

In [ ]:
pressESGdf['ESG_score']=pressESGdf['final_processed_text'].apply(nlp)

In [ ]:
pes = pressESGdf['ESG_score'].apply(lambda x: pd.Series(x[0]))

In [ ]:
pes.columns= ['label_s','score_s']

In [ ]:
pressESGfinal= pd.concat((pressESGdf, pes),axis=1)

In [ ]:
pressESGfinal.head(1)

,symbol,date,title,text,processed_text,final_processed_text,ESG,label,score,ESG_score,label_s,score_s
1,APA,2023-03-01 17:05:00,APA CORPORATION ANNOUNCES 2023 ESG GOALS,"HOUSTON, MARCH 01, 2023 (GLOBE NEWSWIRE) -- AP...","[houston, march, globe, newswire, corporation,...",houston march globe newswire corporation nasda...,"[{'label': 'Environmental', 'score': 0.9437037...",Environmental,0.943704,"[{'label': 'Positive', 'score': 0.703463196754...",Positive,0.703463


In [ ]:
pressESGfinal.to_pickle("pressESGfinal.pkl")

In [ ]:
TransESGfinal = pd.read_pickle("final_trans.pkl")

In [ ]:
TransESGfinal.head(100)[23:24][TransESGfinal.columns[[0,3,8,10,13]]]["final_text"].tolist()

[['continue invest develop technology portfolio hydrogen carbon capture clean power']]

In [ ]:
TransESGfinal.head(100)[23:24][TransESGfinal.columns[[0,3,8,10,13]]]

,symbol,date,final_text,label,label_s
337,BKR,2023-04-19 12:18:04,[continue invest develop technology portfolio ...,Environmental,Positive


In [ ]:
TransESGfinal_score= TransESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
TransESGfinal_score.columns=  ['label_count','senitiment_count', 'score_s']

In [ ]:
pressESGfinal_score= pressESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
pressESGfinal_score.columns= ['label_count','senitiment_count', 'score_s']

In [ ]:
pressESGfinal_score= pressESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
pressESGfinal_score.columns = ['label_count','senitiment_count', 'score_s']

In [ ]:
nwESGfinal_score = nwESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
nwESGfinal_score.columns= ['label_count','senitiment_count', 'score_s']

In [ ]:
pressESGfinal_score.loc[['COP','XOM']]

label_count  senitiment_count   score_s
symbol label         label_s                                          
COP    Environmental Positive            1                 1  0.999067
       Social        Neutral             3                 3  0.863043
XOM    Environmental Neutral            19                19  0.995695
                     Positive            5                 5  0.988097
       Governance    Neutral             3                 3  0.999911
                     Positive            1                 1  0.999260
       Social        Negative            1                 1  0.742412
                     Neutral            12                12  0.982514
                     Positive            3                 3  0.939571

In [ ]:
nwESGfinal_score.loc[['COP','XOM']]

label_count  senitiment_count   score_s
symbol label         label_s                                          
COP    Social        Negative            1                 1  0.978692
                     Neutral            10                10  0.967055
                     Positive            3                 3  0.989896
XOM    Environmental Negative            2                 2  0.818651
                     Neutral             2                 2  0.960986
                     Positive            1                 1  0.999998
       Social        Negative            2                 2  0.966253
                     Neutral             6                 6  0.996266

In [ ]:
TransESGfinal_score.loc[['XOM']]

label_count  senitiment_count   score_s
symbol label  label_s                                          
XOM    Social Neutral             1                 1  0.713562
              Positive            2                 2  0.997577

**calculate ESG score** 

In [ ]:
nwsp500= pd.read_pickle('nwESGfinal_sp500.pkl')

In [ ]:
psp500= pd.read_pickle('pressESGfinal_sp500.pkl')

In [ ]:
tsp500= pd.read_pickle('final_trans.pkl')

In [ ]:
tsp500.shape

(233, 15)

In [ ]:
nwsp500.columns

Index(['symbol', 'publishedDate', 'title', 'site', 'text', 'processed_text',
       'final_processed_text', 'ESG', 'label', 'score', 'ESG_score', 'label_s',
       'score_s'],
      dtype='object')

In [ ]:
psp500[['symbol',  'label', 'score',  'label_s',  'score_s']]

,symbol,label,score,label_s,score_s
2,MMM,Environmental,0.983147,Positive,0.927325
3,MMM,Environmental,0.983382,Neutral,0.811366
4,MMM,Social,0.949386,Positive,1.000000
7,MMM,Social,0.817901,Neutral,0.969614
8,MMM,Governance,0.912281,Positive,0.877227
...,...,...,...,...,...
8,ZBRA,Social,0.891706,Neutral,0.996550
9,ZBRA,Social,0.928836,Positive,0.869265
2,ZTS,Social,0.987772,Neutral,0.997603
3,ZTS,Social,0.954070,Positive,0.512288


In [ ]:
def comdf(df1,df2,df3):
  return pd.concat([df1[['symbol',  'label', 'score',  'label_s',  'score_s']],df2[['symbol',  'label', 'score',  'label_s',  'score_s']], df3[['symbol',  'label', 'score',  'label_s',  'score_s']]],ignore_index=True)

In [ ]:
allsp500= comdf(psp500,nwsp500,tsp500)

In [ ]:
allsp500df= allsp500.groupby(['symbol','label','label_s']).agg( label_count =('label', 'count'),label_S_count=( 'label_s',"count")).reset_index().copy()

In [ ]:
def score(nwsp500df):
  result = np.select(condlist=[nwsp500df['label_s']=='Positive',nwsp500df['label_s']=='Negative',nwsp500df['label_s']=='Neutral'],\
          choicelist=[nwsp500df['label_S_count']*-1,nwsp500df['label_S_count']*1 ,nwsp500df['label_S_count']*-0.5])
  nwsp500df['score'] = result
  return nwsp500df

In [ ]:
score(allsp500df)

,symbol,label,label_s,label_count,label_S_count,score
0,A,Environmental,Positive,1,1,-1.0
1,A,Social,Negative,1,1,1.0
2,A,Social,Neutral,5,5,-2.5
3,A,Social,Positive,6,6,-6.0
4,AAL,Environmental,Positive,1,1,-1.0
...,...,...,...,...,...,...
1483,ZION,Social,Negative,2,2,2.0
1484,ZION,Social,Neutral,5,5,-2.5
1485,ZION,Social,Positive,2,2,-2.0
1486,ZTS,Social,Neutral,2,2,-1.0


In [ ]:
allsp500df

,symbol,label,label_s,label_count,label_S_count,score
0,A,Environmental,Positive,1,1,-1.0
1,A,Social,Negative,1,1,1.0
2,A,Social,Neutral,5,5,-2.5
3,A,Social,Positive,6,6,-6.0
4,AAL,Environmental,Positive,1,1,-1.0
...,...,...,...,...,...,...
1483,ZION,Social,Negative,2,2,2.0
1484,ZION,Social,Neutral,5,5,-2.5
1485,ZION,Social,Positive,2,2,-2.0
1486,ZTS,Social,Neutral,2,2,-1.0


In [ ]:
def agg_score(nwsp500df):
  return nwsp500df.groupby(['symbol','label']).agg(agg_score=('score',"sum"))*1/3

In [ ]:
allsp500df = agg_score(allsp500df)

In [ ]:
def total_score(nwsp500df):
  return nwsp500df.groupby('symbol').agg(total_score=('agg_score',"sum"))

In [ ]:
allsp500df= total_score(allsp500df)

In [ ]:
allsp500df

,total_score
symbol,
A,-2.833333
AAL,-1.333333
AAP,-1.333333
AAPL,-2.666667
ABBV,-5.000000
...,...
YUM,-0.833333
ZBH,0.000000
ZBRA,-1.666667


In [ ]:
from urllib.request import urlopen, Request

import requests

# To extract fundamental data
from bs4 import BeautifulSoup

demo= '97591392fd6d2255da9db4d2b086bac5'
sp_500= requests.get(f'https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={demo}').json()
sp_500df= pd.DataFrame(sp_500)

In [ ]:
nwdf = pd.merge(sp_500df,allsp500df, on='symbol')

NameError: ignored

In [ ]:
#nwdf.iloc[nwdf.groupby('sector')['total_score'].rank(pct=True).index]

In [ ]:
def esg_rank(nwdf):
  nwdf['ESG_score']= nwdf.groupby('sector', as_index=False)['total_score'].rank(pct=True, ascending=False)

In [ ]:
esg_rank(nwdf)

In [ ]:
nwdf.to_pickle("alldf.pkl")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
score=pd.read_pickle("alldf.pkl")

In [ ]:
  df= score[score.columns[[0,1,2,8,9]]]

In [ ]:
df.columns = ['symbol', 'name', 'sector', 'Total_mean_score', 'ESG_score/Percentile_rank']

In [ ]:
df["Total_mean_score"]= df["Total_mean_score"]*-1

<ipython-input-20-768a2d32bf84>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Total_mean_score"]= df["Total_mean_score"]*-1


In [ ]:
df[df['sector']=='Communication Services']

,symbol,name,sector,Total_mean_score,ESG_score/Percentile_rank
5,ATVI,Activision Blizzard,Communication Services,1.833333,0.395833
22,GOOGL,Alphabet Inc. (Class A),Communication Services,5.166667,0.916667
23,GOOG,Alphabet Inc. (Class C),Communication Services,5.500000,0.958333
50,T,AT&T,Communication Services,1.333333,0.250000
104,CHTR,Charter Communications,Communication Services,2.333333,0.500000
121,CMCSA,Comcast,Communication Services,3.666667,0.729167
151,DISH,Dish Network,Communication Services,-1.166667,0.041667
152,DIS,Disney,Communication Services,4.500000,0.854167
169,EA,Electronic Arts,Communication Services,3.500000,0.666667
207,FOXA,Fox Corporation (Class A),Communication Services,1.833333,0.395833


In [ ]:
with pd.option_context('display.max_rows', 100,
                       'display.max_columns', None,
                       'display.width', 1000,
                       'display.precision', 3,
                       'display.colheader_justify', 'center'):
    display(df[df['sector']=='Communication Services'])

,symbol,name,sector,Total_mean_score,ESG_score/Percentile_rank
5,ATVI,Activision Blizzard,Communication Services,1.833,0.396
22,GOOGL,Alphabet Inc. (Class A),Communication Services,5.167,0.917
23,GOOG,Alphabet Inc. (Class C),Communication Services,5.500,0.958
50,T,AT&T,Communication Services,1.333,0.250
104,CHTR,Charter Communications,Communication Services,2.333,0.500
121,CMCSA,Comcast,Communication Services,3.667,0.729
151,DISH,Dish Network,Communication Services,-1.167,0.042
152,DIS,Disney,Communication Services,4.500,0.854
169,EA,Electronic Arts,Communication Services,3.500,0.667
207,FOXA,Fox Corporation (Class A),Communication Services,1.833,0.396


In [ ]:
dff= df[df['sector']=='Communication Services'][['symbol','sector']].merge(allsp500df, on='symbol')

In [ ]:
dff.columns= ['symbol', 'sector', 'label_class', 'label_sentiment', 'sentiment_count', 'label_Sentiment_count','score']

In [ ]:
dff['score']= dff['score']*-1

In [ ]:
dff.head(16)[['symbol', 'sector', 'label_class', 'label_sentiment', 'sentiment_count', 'score']]

,symbol,sector,label_class,label_sentiment,sentiment_count,score
0,ATVI,Communication Services,Environmental,Negative,1,-1.0
1,ATVI,Communication Services,Social,Negative,2,-2.0
2,ATVI,Communication Services,Social,Neutral,15,7.5
3,ATVI,Communication Services,Social,Positive,1,1.0
4,GOOGL,Communication Services,Social,Negative,2,-2.0
5,GOOGL,Communication Services,Social,Neutral,21,10.5
6,GOOGL,Communication Services,Social,Positive,7,7.0
7,GOOG,Communication Services,Social,Negative,1,-1.0
8,GOOG,Communication Services,Social,Neutral,21,10.5
9,GOOG,Communication Services,Social,Positive,7,7.0
